**Toxic Comment Classification Assessment**

**The Jigsaw Toxic Comment** Classification dataset is a collection of Wikipedia talk page comments, labeled for various types of toxicity (toxic, severe_toxic, obscene, threat, insult, identity_hate). The goal of this task is to build a safety model capable of flagging abusive content.

The labels are originally multi-label (a comment can be both an insult and a threat). For this project i want just to generalise and know if its abusive or safe , we aggregate these into a Binary Classification task: Safe (0) vs. Abusive (1). This focuses the model on the detection of general malicious intent, which is critical for content moderation systems.

link to model : https://www.kaggle.com/competitions/jigsaw-toxic-comment-classification-challenge/data

In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import re
import string
import torch
from sklearn.model_selection import train_test_split

# Load Data
try:
    df = pd.read_csv('data/train.csv', nrows=50000) # Using 50k subset for dev
    print(f"Data Loaded: {len(df)} rows")
except:
    print("Error loading data")

# Pivot to Binary Target
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
df['is_abusive'] = df[label_cols].max(axis=1)

x = df['comment_text']
y = df['is_abusive']

print(f"Sample Text: {x[0]}")
print(f"Label: {y[0]}")

Data Loaded: 50000 rows
Sample Text: Explanation
Why the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27
Label: 0
